In [1]:
from typing import Dict
from tqdm.auto import tqdm

import numpy as np
import random
import pandas as pd
import ydata_profiling

import scienceplots
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

import torch
import os

%matplotlib inline

pd.plotting.register_matplotlib_converters()
sns.set_style("whitegrid")
sns.set_palette("RdBu")
sns.set(
    rc={'text.usetex': True},
    font="serif",
    font_scale=1.2
)

# days = {i: pd.read_pickle(f'../Datasets/day{str(i).zfill(2)}.pkl')
#         for i in range(1, 22)}


In [2]:
feature = pd.read_pickle('../Datasets/feature_duration_sum.pkl')


In [10]:
df = pd.DataFrame(feature.pivot_table(
    index='uid',
    values='a',
    columns='day',
).fillna(0).to_records())


,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1419.0,0.0,534.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2143.0,81.0,209.0,3495.0
3,50.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
47340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47342,0.0,0.0,676.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47343,278.0,206.0,5981.0,1061.0,865.0,479.0,135.0,1.0,227.0,1612.0,0.0


In [25]:
X_train, y_train = df[[str(i) for i in range(1, 11)]].values, df['11'].apply(
    lambda x: 1 if x > 0 else 0).values


In [26]:
X_train


array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 1.419e+03,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 2.143e+03, 8.100e+01,
        2.090e+02],
       ...,
       [0.000e+00, 0.000e+00, 6.760e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.780e+02, 2.060e+02, 5.981e+03, ..., 1.000e+00, 2.270e+02,
        1.612e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [27]:
y_train


array([0, 1, 1, ..., 0, 0, 0])

In [28]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=20, random_state=71)
model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=20, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=71, ...)

In [29]:
X_test, y_test = df[[str(i) for i in range(2, 12)]].values, df['12'].apply(
    lambda x: 1 if x > 0 else 0).values


In [30]:
pred = model.predict_proba(X_test)[:, 1]
pred_label = np.where(pred > 0.5, 1, 0)


In [31]:
pred_label


array([0, 0, 1, ..., 0, 1, 0])

In [32]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred_label)


0.8835146266765234

In [37]:
def NMSE(y_pred, y_true):
    return (((y_pred-y_true)**2).sum()/((y_pred-np.mean(y_true))**2).sum())**(1/2)


In [36]:
y_i = np.array([1, 2, 1, 0])
y_hat = np.array([0, 0, 0, 1])

NMSE(y_i, y_hat)


7

# 回归问题

In [38]:
X_train, y_train = df[[str(i) for i in range(1, 11)]].values, df['11'].values
X_test, y_test = df[[str(i) for i in range(2, 12)]].values, df['12'].values


In [42]:
import xgboost as xg

xgb_r = xg.XGBRegressor(n_estimators=10, seed=123)
xgb_r.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [43]:
y_pred = xgb_r.predict(X_test)


In [44]:
y_pred


array([ 15.689838, 337.77155 , 984.31964 , ...,  60.692284, 495.586   ,
        15.689838], dtype=float32)

In [45]:
NMSE(y_pred, y_test)


1.4706247228722296